# 1. Setup

In [1]:
%run run_finetune.py

In [2]:
import sys
import argparse


cmd_str = """run_finetune.py 
--model_type dna 
--tokenizer_name=dna6    
--model_name_or_path /mnt/ceph/users/zzhang/DNABERT/myExp/6-new-12w-0     
--task_name dnaprom     
--do_train     
--do_eval     
--data_dir ./sample_data/ft/6     
--max_seq_length 100     
--per_gpu_eval_batch_size=32       
--per_gpu_train_batch_size=32       
--learning_rate 2e-4     
--num_train_epochs 5.0     
--output_dir ./outputs/test_ft_6   
--evaluate_during_training     
--logging_steps 100     
--save_steps 4000     
--warmup_percent 0.1     
--hidden_dropout_prob 0.1     
--overwrite_output     
--weight_decay 0.01     
--n_process 8
"""

arg_list = cmd_str.split()
sys.argv = arg_list
print("python " + " ".join(cmd_str.split()))

python run_finetune.py --model_type dna --tokenizer_name=dna6 --model_name_or_path /mnt/ceph/users/zzhang/DNABERT/myExp/6-new-12w-0 --task_name dnaprom --do_train --do_eval --data_dir ./sample_data/ft/6 --max_seq_length 100 --per_gpu_eval_batch_size=32 --per_gpu_train_batch_size=32 --learning_rate 2e-4 --num_train_epochs 5.0 --output_dir ./outputs/test_ft_6 --evaluate_during_training --logging_steps 100 --save_steps 4000 --warmup_percent 0.1 --hidden_dropout_prob 0.1 --overwrite_output --weight_decay 0.01 --n_process 8


In [3]:
args = parser()
args

Namespace(adam_epsilon=1e-08, attention_probs_dropout_prob=0.1, beta1=0.9, beta2=0.999, cache_dir='', config_name='', data_dir='./sample_data/ft/6', device=device(type='cuda'), do_ensemble_pred=False, do_eval=True, do_lower_case=False, do_predict=False, do_train=True, do_visualize=False, early_stop=0, eval_all_checkpoints=False, evaluate_during_training=True, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=1, hidden_dropout_prob=0.1, learning_rate=0.0002, local_rank=-1, logging_steps=100, max_grad_norm=1.0, max_seq_length=100, max_steps=-1, model_name_or_path='/mnt/ceph/users/zzhang/DNABERT/myExp/6-new-12w-0', model_type='dna', n_gpu=1, n_process=8, no_cuda=False, num_rnn_layer=2, num_train_epochs=5.0, output_dir='./outputs/test_ft_6', overwrite_cache=False, overwrite_output_dir=True, per_gpu_eval_batch_size=32, per_gpu_pred_batch_size=8, per_gpu_train_batch_size=32, predict_dir=None, predict_scan_size=1, result_dir=None, rnn='lstm', rnn_dropout=0.0, rnn_hidden=768, save_s

In [4]:
# Set seed
set_seed(args)

# Prepare GLUE task
args.task_name = args.task_name.lower()
if args.task_name not in processors:
    raise ValueError("Task not found: %s" % (args.task_name))
processor = processors[args.task_name]()
args.output_mode = output_modes[args.task_name]
label_list = processor.get_labels()
num_labels = len(label_list)

In [5]:
label_list, output_modes

(['0', '1'],
 {'cola': 'classification',
  'mnli': 'classification',
  'mnli-mm': 'classification',
  'mrpc': 'classification',
  'sst-2': 'classification',
  'sts-b': 'regression',
  'qqp': 'classification',
  'qnli': 'classification',
  'rte': 'classification',
  'wnli': 'classification',
  'dnaprom': 'classification',
  'dna690': 'classification',
  'dnapair': 'classification',
  'dnasplice': 'classification'})

# 2. Get DNABERT model

In [6]:
# FZZ: get model from MODEL_CLASSES
args.model_type = args.model_type.lower()
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]

config = config_class.from_pretrained(
    args.config_name if args.config_name else args.model_name_or_path,
    num_labels=num_labels,
    finetuning_task=args.task_name,
    cache_dir=args.cache_dir if args.cache_dir else None,
)

config.hidden_dropout_prob = args.hidden_dropout_prob
config.attention_probs_dropout_prob = args.attention_probs_dropout_prob
if args.model_type in ["dnalong", "dnalongcat"]:
    assert args.max_seq_length % 512 == 0
config.split = int(args.max_seq_length/512)
config.rnn = args.rnn
config.num_rnn_layer = args.num_rnn_layer
config.rnn_dropout = args.rnn_dropout
config.rnn_hidden = args.rnn_hidden

tokenizer = tokenizer_class.from_pretrained(
    args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
    do_lower_case=args.do_lower_case,
    cache_dir=args.cache_dir if args.cache_dir else None,
)
model = model_class.from_pretrained(
    args.model_name_or_path,
    from_tf=bool(".ckpt" in args.model_name_or_path),
    config=config,
    cache_dir=args.cache_dir if args.cache_dir else None,
)

<class 'transformers.tokenization_dna.DNATokenizer'>


In [7]:
# main workhorse for fine-tuning classification/regression tasks
?model

In [8]:
# load datasets
print(args.data_dir)
train_dataset = load_and_cache_examples(args, args.task_name, tokenizer, evaluate=False)

./sample_data/ft/6


In [9]:
# convert datasets to torch.DataSet
args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

In [10]:
# get a batch from `torch.DataLoader`
batch = next(iter(train_dataloader))
inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}

In [11]:
# outputs is tuple of (Loss, Logits)
outputs = model(**inputs)
print(len(outputs))
[outputs[0], outputs[1][0:5]]

2


[tensor(0.6802, grad_fn=<NllLossBackward>),
 tensor([[ 0.2693, -0.0378],
         [ 0.4302,  0.5268],
         [ 0.1384, -0.0703],
         [ 0.2343, -0.0050],
         [ 0.1529, -0.1760]], grad_fn=<SliceBackward>)]

# 3. Features are embedded prior in `load_and_cache_examples`

In [12]:
examples = processor.get_train_examples(args.data_dir)
print(len(examples))
examples[0]

32366


{
  "guid": "train-1",
  "label": "0",
  "text_a": "CACAGC ACAGCC CAGCCA AGCCAG GCCAGC CCAGCC CAGCCA AGCCAC GCCACT CCACTA CACTAG ACTAGG CTAGGC TAGGCC AGGCCC GGCCCT GCCCTG CCCTGT CCTGTC CTGTCC TGTCCT GTCCTG TCCTGC CCTGCA CTGCAG TGCAGC GCAGCC CAGCCC AGCCCC GCCCCC CCCCCT CCCCTG CCCTGT CCTGTA CTGTAG TGTAGG GTAGGG TAGGGG AGGGGT GGGGTC GGGTCT GGTCTG GTCTGG TCTGGA CTGGAA TGGAAC GGAACA GAACAG AACAGC ACAGCC CAGCCA AGCCAG GCCAGG CCAGGA CAGGAG AGGAGT GGAGTG GAGTGG AGTGGT GTGGTT TGGTTT GGTTTA GTTTAA TTTAAG TTAAGA TAAGAG AAGAGG AGAGGC GAGGCA AGGCAG GGCAGG GCAGGG CAGGGG AGGGGA GGGGAG GGGAGT GGAGTC GAGTCG AGTCGC GTCGCC TCGCCT CGCCTT GCCTTG CCTTGC CTTGCC TTGCCC TGCCCT GCCCTG CCCTGT CCTGTG CTGTGC TGTGCC GTGCCA TGCCAC GCCACA CCACAC",
  "text_b": null
}

In [13]:
max_length = args.max_seq_length
pad_on_left = bool(args.model_type in ["xlnet"])
pad_token = tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0]
pad_token_segment_id = 4 if args.model_type in ["xlnet"] else 0

In [14]:
output_mode = output_modes[args.task_name]
print(output_mode)
features = convert_examples_to_features(
    examples,
    tokenizer,
    label_list=label_list,
    max_length=max_length,
    output_mode=output_mode,
    pad_on_left=pad_on_left,  # pad on the left for xlnet
    pad_token=pad_token,
    pad_token_segment_id=pad_token_segment_id,)

classification


In [15]:
print("features ", len(features))
print("examples ", len(examples))
print(len(features[0].attention_mask))

features  32366
examples  32366
100


In [16]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": "dnaprom",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "num_return_sequences": 1,
  "num_rnn_layer": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pruned_heads": {},
  "repetition_penalty": 1.0,
  "rnn": "lstm",
  "rnn_dropout": 0.0,
  "rnn_hidden": 768,
  "split": 0,
  "temperature": 1.0,
  